In [1]:
import matplotlib
matplotlib.use("Agg")

# impor paket yang diperlukan
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, RepeatVector, Masking, TimeDistributed
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import plot_model

from imutils import paths
from cv2 import cv2
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import os
import warnings

In [2]:
warnings.filterwarnings('ignore')
'''
# membangun parser argumen dan parsing argumen
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True,
	help="path to input dataset of images")
ap.add_argument("-m", "--model", required=True,
	help="path to output trained model")
ap.add_argument("-l", "--label-bin", required=True,
	help="path to output label binarizer")
ap.add_argument("-p", "--plot", required=True,
	help="path to output accuracy/loss plot")
args = vars(ap.parse_args())
'''
# inisialisasi data dan label
print("[INFO] loading images...")
data = []
labels = []

[INFO] loading images...


In [3]:
# ambil jalur gambar dan merandom gambar secara acak
imagePaths = sorted(list(paths.list_images('new-dataset')))
random.seed(42)
random.shuffle(imagePaths)

In [5]:
# loop di atas gambar input
for imagePath in imagePaths:
	# memuat gambar, mengubah ukuran gambar menjadi 32x32 piksel (mengabaikan aspek rasio), 
	# meratakan gambar menjadi 32x32x3 = 3072 piksel gambar ke dalam daftar, 
	# dan menyimpan gambar dalam daftar data
	print("[INFO] Preproccessing image: ", imagePath)
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (32, 32)).flatten()
	data.append(image)

	# ekstrak label kelas dari jalur gambar dan memperbarui daftar label
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

# skala intensitas piksel mentah ke kisaran [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# mempartisi data menjadi pelatihan dan pengujian split menggunakan 75% dari data untuk pelatihan 
# dan sisanya 25% untuk pengujian
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

# mengonversi label dari bilangan bulat ke vektor (untuk klasifikasi biner 2-kelas, 
# disini harus menggunakan fungsi 'to_categorical karena LabelBinarizer 
# dari scikit-learn tidak akan mengembalikan vektor)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

[INFO] Preproccessing image:  new-dataset\PS\(645).jpg
[INFO] Preproccessing image:  new-dataset\PS\(637).jpg
[INFO] Preproccessing image:  new-dataset\NOT PLASTIC\(226).jpg
[INFO] Preproccessing image:  new-dataset\OTHERS\(238).jpg
[INFO] Preproccessing image:  new-dataset\PP\(159).jpg
[INFO] Preproccessing image:  new-dataset\OTHERS\(252).jpg
[INFO] Preproccessing image:  new-dataset\PP\(63).jpg
[INFO] Preproccessing image:  new-dataset\PS\(98).jpg
[INFO] Preproccessing image:  new-dataset\OTHERS\(77).jpg
[INFO] Preproccessing image:  new-dataset\PS\(646).jpg
[INFO] Preproccessing image:  new-dataset\OTHERS\(116).jpg
[INFO] Preproccessing image:  new-dataset\PETE\(274).jpg
[INFO] Preproccessing image:  new-dataset\PS\(670).jpg
[INFO] Preproccessing image:  new-dataset\NOT PLASTIC\(496).jpg
[INFO] Preproccessing image:  new-dataset\HDPE\HDPE.30.jpg
[INFO] Preproccessing image:  new-dataset\PS\(667).jpg
[INFO] Preproccessing image:  new-dataset\LDPE\(329).jpg
[INFO] Preproccessing imag

In [6]:
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

In [7]:
INIT_LR = 0.01
EPOCHS = 1500

# kompilasi model menggunakan SGD sebagai pengoptimal dan categorical 
# cross-entropy loss (menggunakan binary_crossentropy untuk klasifikasi 2 kelas)
# dimana SGD adalah pengoptimalan library yang menggunakan backpropagation
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# melatih jaringan saraf
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=EPOCHS, batch_size=32)

# mengevaluasi jaringan
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))



[INFO] training network...
Epoch 1/1500
26/26 [==============================] - 2s 43ms/step - loss: 1.9825 - accuracy: 0.2276 - val_loss: 1.8849 - val_accuracy: 0.3297
Epoch 2/1500
26/26 [==============================] - 1s 30ms/step - loss: 1.9553 - accuracy: 0.2349 - val_loss: 1.9073 - val_accuracy: 0.2681
Epoch 3/1500
26/26 [==============================] - 1s 32ms/step - loss: 1.9577 - accuracy: 0.2167 - val_loss: 1.8922 - val_accuracy: 0.3297
Epoch 4/1500
26/26 [==============================] - 1s 34ms/step - loss: 1.9501 - accuracy: 0.2324 - val_loss: 1.8859 - val_accuracy: 0.3297
Epoch 5/1500
26/26 [==============================] - 1s 31ms/step - loss: 1.9457 - accuracy: 0.2385 - val_loss: 1.8726 - val_accuracy: 0.3297
Epoch 6/1500
26/26 [==============================] - 1s 31ms/step - loss: 1.9434 - accuracy: 0.2482 - val_loss: 1.8686 - val_accuracy: 0.3297
Epoch 7/1500
26/26 [==============================] - 1s 32ms/step - loss: 1.9384 - accuracy: 0.2470 - val_loss: 1.

In [8]:
# merencanakan kehilangan dan akurasi pelatihan
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (Backpropagation NN)")
plt.xlabel("Epoch " + str(EPOCHS))
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('model/plastic_plot.png')



In [9]:
# merencanakan kehilangan dan akurasi pelatihan
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training and Val Loss")
plt.xlabel("Epoch " + str(EPOCHS))
plt.ylabel("Loss")
plt.legend()
plt.savefig('model/plastic_plot_loss.png')


In [10]:
# merencanakan kehilangan dan akurasi pelatihan
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training and Val Accuracy")
plt.xlabel("Epoch " + str(EPOCHS))
plt.ylabel("Accuracy")
plt.legend()
plt.savefig('model/plastic_plot_accuracy.png')

In [11]:
# simpan model dan beri label binarizer ke disk
print("[INFO] serializing network and label binarizer...")
model.save('model/plastic.model')
f = open('model/plastic_lb.pickle', "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network and label binarizer...
INFO:tensorflow:Assets written to: model/plastic.model\assets
